In [1]:
!pip install unidecode

In [2]:
# Iniciando Sessão Spark
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .appName('Votacao_Candidato_MunZona_2022') \
    .getOrCreate()

In [3]:
# Lendo o CSV e Salvando em um DataFrame
df = spark.read.csv("jobs/votacao_candidato_munzona_2022_BRASIL.csv", sep=";", header=True, inferSchema=True, encoding="ISO-8859-1")
df.show(20)

+----------+-------------------+-----------+---------------+-----------------+--------+----------+--------------------+----------+--------------+-----+-----+-----------------+------------+--------------------+-------+--------+-----------------+------------+------------+--------------------+--------------------+-------------------+-----------------------+-----------------------+------------------------+------------------------+---------------+----------+-------------+--------------------+------------+--------------------+-------------+-----------------------+------------+---------------+-----------------------+-------------------+-----------------+------------------------+-------------------------+----------------+----------------+
|DT_GERACAO|         HH_GERACAO|ANO_ELEICAO|CD_TIPO_ELEICAO|  NM_TIPO_ELEICAO|NR_TURNO|CD_ELEICAO|          DS_ELEICAO|DT_ELEICAO|TP_ABRANGENCIA|SG_UF|SG_UE|            NM_UE|CD_MUNICIPIO|        NM_MUNICIPIO|NR_ZONA|CD_CARGO|         DS_CARGO|SQ_CANDIDATO|NR

In [4]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
import unidecode

# Definindo a função udf para remover acentos
udf_remove_acentos = udf(lambda x: unidecode.unidecode(x) if x else None, StringType())

# Selecionando as colunas necessárias para o OBT
columns = ["NR_TURNO","DS_ELEICAO","TP_ABRANGENCIA","SG_UF", "NM_UE","NM_MUNICIPIO","NR_ZONA","DS_CARGO","NR_CANDIDATO","NM_CANDIDATO","NM_URNA_CANDIDATO","DS_SITUACAO_CANDIDATURA","NR_PARTIDO","SG_PARTIDO","NM_PARTIDO","NM_COLIGACAO","DS_COMPOSICAO_COLIGACAO","ST_VOTO_EM_TRANSITO","QT_VOTOS_NOMINAIS","NM_TIPO_DESTINACAO_VOTOS","QT_VOTOS_NOMINAIS_VALIDOS","DS_SIT_TOT_TURNO"]

# Selecionando as colunas e aplicando a remoção de acentos à coluna "NM_UE"
new_df = df.select(columns).withColumn("NM_UE", udf_remove_acentos("NM_UE"))

new_df.show(20)

+--------+--------------------+--------------+-----+-----------------+--------------------+-------+-----------------+------------+--------------------+--------------------+-----------------------+----------+-------------+--------------------+---------------+-----------------------+-------------------+-----------------+------------------------+-------------------------+----------------+
|NR_TURNO|          DS_ELEICAO|TP_ABRANGENCIA|SG_UF|            NM_UE|        NM_MUNICIPIO|NR_ZONA|         DS_CARGO|NR_CANDIDATO|        NM_CANDIDATO|   NM_URNA_CANDIDATO|DS_SITUACAO_CANDIDATURA|NR_PARTIDO|   SG_PARTIDO|          NM_PARTIDO|   NM_COLIGACAO|DS_COMPOSICAO_COLIGACAO|ST_VOTO_EM_TRANSITO|QT_VOTOS_NOMINAIS|NM_TIPO_DESTINACAO_VOTOS|QT_VOTOS_NOMINAIS_VALIDOS|DS_SIT_TOT_TURNO|
+--------+--------------------+--------------+-----+-----------------+--------------------+-------+-----------------+------------+--------------------+--------------------+-----------------------+----------+-------------+-

In [5]:
# Recebendo as variáveis do Banco
hostname_or_ip = "postgres"
port = "5432"
db = "obt"
user = "metabase"
password = "FlaDWaiNEdySeRiG"
schema = "obt"
table = "obt"

In [6]:
# Ajustando as variáveis do formulário
url = "jdbc:postgresql://" + hostname_or_ip + ":" + port + "/" + db
properties = {
    "user": user,
    "password": password,
    "driver": "org.postgresql.Driver"
}

In [7]:
# Gravar dados no banco postgresql
new_df.write.jdbc(url=url, table=table, mode="overwrite", properties=properties)